In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pdb
import json
import glob
from natsort import natsorted
import kipoiseq

import basenji
from basenji import dataset
import tensorflow as tf
import tensorflow_probability as tfp

from promoter_modelling.utils import fasta_utils

2024-02-18 14:17:51.240237: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-18 14:17:51.352864: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-18 14:17:51.352914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-18 14:17:51.354874: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-18 14:17:51.369195: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# TFRecord constants
TFR_INPUT = 'sequence'
TFR_OUTPUT = 'target'

def file_to_records(filename):
    return tf.data.TFRecordDataset(filename, compression_type='ZLIB')

def generate_parser(seq_length, seq_1hot, seq_depth, seq_length_crop, target_length, num_targets, with_preds=False, raw=False):
    def parse_proto(example_protos):
        """Parse TFRecord protobuf."""

        # define features
        features = {
            TFR_INPUT: tf.io.FixedLenFeature([], tf.string),
            TFR_OUTPUT: tf.io.FixedLenFeature([], tf.string)
        }
        if with_preds:
            features = {
            TFR_INPUT: tf.io.FixedLenFeature([], tf.string),
            TFR_OUTPUT: tf.io.FixedLenFeature([], tf.string),
            "cross2020_prediction": tf.io.FixedLenFeature([], tf.string),
        }

        # parse example into features
        parsed_features = tf.io.parse_single_example(example_protos, features=features)

        # decode sequence
        sequence = tf.io.decode_raw(parsed_features[TFR_INPUT], tf.uint8)
        if not raw:
            if seq_1hot:
                sequence = tf.reshape(sequence, [seq_length])
                sequence = tf.one_hot(sequence, 1+seq_depth, dtype=tf.uint8)
                sequence = sequence[:,:-1] # drop N
            else:
                sequence = tf.reshape(sequence, [seq_length, seq_depth])
            if seq_length_crop is not None:
                crop_len = (seq_length - seq_length_crop) // 2
                sequence = sequence[crop_len:-crop_len,:]
            sequence = tf.cast(sequence, tf.float32)

        # decode targets
        targets = tf.io.decode_raw(parsed_features[TFR_OUTPUT], tf.float16)
        if not raw:
            targets = tf.reshape(targets, [target_length, num_targets])
            targets = tf.cast(targets, tf.float32)
        
        if with_preds:
            # decode targets
            cross2020_prediction = tf.io.decode_raw(parsed_features["cross2020_prediction"], tf.float16)
            if not raw:
                cross2020_prediction = tf.reshape(cross2020_prediction, [target_length, num_targets])
                cross2020_prediction = tf.cast(cross2020_prediction, tf.float32)
            return sequence, targets, cross2020_prediction

        return sequence, targets

    return parse_proto

In [3]:
def feature_bytes(values):
    """Convert numpy arrays to bytes features."""
    values = values.flatten().tobytes()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))

def serialize_example(sequence, target, cross2020_prediction=None):
    """
    Creates a tf.train.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.train.Example-compatible
    # data type.
    if cross2020_prediction is not None:
        feature = {
          'sequence': feature_bytes(sequence),
          'target': feature_bytes(target),
          'cross2020_prediction': feature_bytes(cross2020_prediction.astype("float16")),
        }
    else:
        feature = {
          'sequence': feature_bytes(sequence),
          'target': feature_bytes(target),
        }

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
# the default Pytorch dataloader for TFRecords doesn't support compressed TFRecords that are created
# by the following code. Therefore, it has been commented out.

# root_data_dir = "/clusterfs/nilah/richard/basenji2/data_pretrained_basenji2"
# enformer_root_data_dir = "/clusterfs/nilah/enformer_data/"
# split_labels = ["train", "test", "valid"]

# ind_to_base = np.array(["A", "C", "G", "T"])
# enformer_input_size = 196608

# for i, species in enumerate(["human", "mouse"]):
#     print(species)
    
#     data_dir = os.path.join(root_data_dir, species)
    
#     if species == "human":
#         fasta_file = "/clusterfs/nilah/aniketh/hg38/hg38.fa"
#         chromosome_sizes = "/global/home/users/aniketh/basenji_CL_trial/data/human.hg38.genome"
#     elif species == "mouse":
#         fasta_file = "/clusterfs/nilah/aniketh/mm10/mm10.fa"
#         chromosome_sizes = "/global/home/users/aniketh/basenji_CL_trial/data/mouse.mm10.genome"
        
#     fasta_extractor = fasta_utils.FastaStringExtractor(fasta_file)
#     chromosome_sizes = pd.read_csv(chromosome_sizes, sep="\t", names=["chromosome", "size"], header=None)
    
#     # create new enformer data dirs
#     enformer_species_data_dir = os.path.join(enformer_root_data_dir, species)
#     os.makedirs(enformer_species_data_dir, exist_ok=True)
#     os.makedirs(os.path.join(enformer_species_data_dir, "tfrecords"), exist_ok=True)
    
#     # read data parameters
#     data_stats_file = '%s/statistics.json' % data_dir
#     with open(data_stats_file) as data_stats_open:
#         data_stats = json.load(data_stats_open)

#     seq_length = data_stats['seq_length']
#     seq_depth = data_stats.get('seq_depth',4)
#     seq_1hot = data_stats.get('seq_1hot', False)
#     target_length = data_stats['target_length']
#     num_targets = data_stats['num_targets']
#     pool_width = data_stats['pool_width']
    
#     # enformer data parameters
#     enformer_data_stats_file = '%s/statistics.json' % enformer_species_data_dir
#     enformer_data_stats = data_stats.copy()
#     enformer_data_stats['seq_length'] = enformer_input_size
    
#     with open(enformer_data_stats_file, "w+") as enformer_data_stats_open:
#         json.dump(enformer_data_stats, enformer_data_stats_open, indent=4)
    
#     # read sequences BED file
#     all_sequences = pd.read_csv(os.path.join(root_data_dir, species, "sequences.bed"), 
#                                 sep="\t", 
#                                 names=["chromosome", "start", "end", "split"], 
#                                 header=None)
    
#     # copy it over to enformer data dir
#     os.system("cp {} {}".format(os.path.join(root_data_dir, species, "sequences.bed"), os.path.join(enformer_root_data_dir, species, "sequences.bed")))
#     # also copy over targets info
#     os.system("cp {} {}".format(os.path.join(root_data_dir, species, "targets.txt"), os.path.join(enformer_root_data_dir, species, "targets.txt")))
    
#     # compute extra sequence to be extracted from each side of original sequence
#     extra_seq_needed = (enformer_data_stats['seq_length'] - seq_length) // 2
    
#     for split_label in split_labels:
#         print(split_label)
        
#         subset_all_sequences = all_sequences[all_sequences["split"] == split_label].reset_index(drop=True)

#         # collect tfrecords
#         tfr_path = '%s/tfrecords/%s-*.tfr' % (data_dir, split_label)
#         tfr_files = natsorted(glob.glob(tfr_path))
        
#         total_rec_count = 0
#         for f in tfr_files:
#             print(f)
            
#             new_f = f.replace(data_dir, enformer_species_data_dir)
#             fcn = f.split("-")[-1].split(".")[0]

#             if os.path.exists(new_f):
#                 print("TFRecords already exist, not rewriting...")
            
#             file = tf.data.TFRecordDataset(f, compression_type='ZLIB').map(generate_parser(seq_length, seq_1hot, seq_depth, None, target_length, num_targets))
            
#             check = True
#             if not os.path.exists(new_f):
#                 new_file = tf.io.TFRecordWriter(new_f, 'ZLIB')
#                 check = False
            
#             for record in tqdm(file.batch(1)):
#                 sequence, target = record
#                 sequence = sequence.numpy().astype("uint8")
#                 target = target.numpy().astype("float16")
                
#                 # assert that genomic locus from sequences BED file matches the sequence in this record
#                 row = subset_all_sequences.iloc[total_rec_count]
#                 sequence_from_fasta = fasta_utils.get_sequence(row["chromosome"], \
#                                                                row["start"] + 1, \
#                                                                row["end"] + 1, \
#                                                                ".", \
#                                                                fasta_extractor)
#                 sequence_from_fasta = np.array([_ for _ in sequence_from_fasta])
#                 sequence_from_one_hot = ind_to_base[sequence[0].argmax(axis=-1)]
#                 N_mask = sequence_from_fasta == "N"
#                 assert np.all(sequence_from_one_hot[~N_mask] == sequence_from_fasta[~N_mask])
                
#                 # now get expanded sequence
#                 # left pad if needed
#                 expanded_sequence_start = max(1, row["start"] + 1 - extra_seq_needed)
#                 if expanded_sequence_start == 1:
#                     left_pad = "".join(["N" for _ in range(1 - (row["start"] + 1 - extra_seq_needed))])
#                 else:
#                     left_pad = ""                    
                
#                 # right pad if needed
#                 chr_size = chromosome_sizes[chromosome_sizes["chromosome"] == row["chromosome"]].iloc[0]["size"]
#                 expanded_sequence_end = min(chr_size + 1, row["end"] + 1 + extra_seq_needed)
#                 if expanded_sequence_end == (chr_size + 1):
#                     right_pad = "".join(["N" for _ in range((row["end"] + 1 + extra_seq_needed) - (chr_size + 1))])
#                 else:
#                     right_pad = ""                
                
#                 expanded_sequence = fasta_utils.get_sequence(row["chromosome"], \
#                                                              expanded_sequence_start, \
#                                                              expanded_sequence_end, \
#                                                              ".", \
#                                                              fasta_extractor)
#                 expanded_sequence = left_pad + expanded_sequence + right_pad
#                 assert len(expanded_sequence) == enformer_input_size
                
#                 # one-hot encode it
#                 expanded_sequence = np.array([i.upper() for i in expanded_sequence])
#                 expanded_sequence_1hot = np.zeros((len(expanded_sequence), 4), dtype="uint8")
#                 expanded_sequence_1hot[expanded_sequence == "A", 0] = 1
#                 expanded_sequence_1hot[expanded_sequence == "C", 1] = 1
#                 expanded_sequence_1hot[expanded_sequence == "G", 2] = 1
#                 expanded_sequence_1hot[expanded_sequence == "T", 3] = 1
#                 expanded_sequence_1hot = expanded_sequence_1hot.reshape(1, len(expanded_sequence), 4)
                                
#                 if not check:
#                     new_file.write(serialize_example(expanded_sequence_1hot, target))
                
#                 total_rec_count += 1

In [4]:
root_data_dir = "/clusterfs/nilah/richard/basenji2/data_pretrained_basenji2"
enformer_root_data_dir = "/global/scratch/users/aniketh/enformer_data/"
split_labels = ["train", "test", "valid"]

ind_to_base = np.array(["A", "C", "G", "T"])
enformer_input_size = 196608

for i, species in enumerate(["human", "mouse"]):
    print(species)
    
    data_dir = os.path.join(root_data_dir, species)
    
    if species == "human":
        fasta_file = "/clusterfs/nilah/aniketh/hg38/hg38.fa"
        chromosome_sizes = "/global/home/users/aniketh/basenji_CL_trial/data/human.hg38.genome"
    elif species == "mouse":
        fasta_file = "/clusterfs/nilah/aniketh/mm10/mm10.fa"
        chromosome_sizes = "/global/home/users/aniketh/basenji_CL_trial/data/mouse.mm10.genome"
        
    fasta_extractor = fasta_utils.FastaStringExtractor(fasta_file)
    chromosome_sizes = pd.read_csv(chromosome_sizes, sep="\t", names=["chromosome", "size"], header=None)
    
    # create new enformer data dirs
    enformer_species_data_dir = os.path.join(enformer_root_data_dir, species)
    os.makedirs(enformer_species_data_dir, exist_ok=True)
    os.makedirs(os.path.join(enformer_species_data_dir, "tfrecords"), exist_ok=True)
    
    # read data parameters
    data_stats_file = '%s/statistics.json' % data_dir
    with open(data_stats_file) as data_stats_open:
        data_stats = json.load(data_stats_open)

    seq_length = data_stats['seq_length']
    seq_depth = data_stats.get('seq_depth',4)
    seq_1hot = data_stats.get('seq_1hot', False)
    target_length = data_stats['target_length']
    num_targets = data_stats['num_targets']
    pool_width = data_stats['pool_width']
    
    # enformer data parameters
    enformer_data_stats_file = '%s/statistics.json' % enformer_species_data_dir
    enformer_data_stats = data_stats.copy()
    enformer_data_stats['seq_length'] = enformer_input_size
    
    with open(enformer_data_stats_file, "w+") as enformer_data_stats_open:
        json.dump(enformer_data_stats, enformer_data_stats_open, indent=4)
    
    # read sequences BED file
    all_sequences = pd.read_csv(os.path.join(root_data_dir, species, "sequences.bed"), 
                                sep="\t", 
                                names=["chromosome", "start", "end", "split"], 
                                header=None)
    
    # copy it over to enformer data dir
    os.system("cp {} {}".format(os.path.join(root_data_dir, species, "sequences.bed"), os.path.join(enformer_root_data_dir, species, "sequences.bed")))
    # also copy over targets info
    os.system("cp {} {}".format(os.path.join(root_data_dir, species, "targets.txt"), os.path.join(enformer_root_data_dir, species, "targets.txt")))
    
    # compute extra sequence to be extracted from each side of original sequence
    extra_seq_needed = (enformer_data_stats['seq_length'] - seq_length) // 2
    
    for split_label in split_labels:
        print(split_label)
        
        subset_all_sequences = all_sequences[all_sequences["split"] == split_label].reset_index(drop=True)

        # collect tfrecords
        tfr_path = '%s/tfrecords/%s-*.tfr' % (data_dir, split_label)
        tfr_files = natsorted(glob.glob(tfr_path))
        
        total_rec_count = 0
        for f in tfr_files:
            print(f)
            
            new_f = f.replace(data_dir, enformer_species_data_dir)
            fcn = f.split("-")[-1].split(".")[0]

            if os.path.exists(new_f):
                print("TFRecords already exist, not rewriting...")
            
            file = tf.data.TFRecordDataset(f, compression_type='ZLIB').map(generate_parser(seq_length, seq_1hot, seq_depth, None, target_length, num_targets))
            
            check = True
            if not os.path.exists(new_f):
                new_file = tf.io.TFRecordWriter(new_f)
                check = False
            
            for record in tqdm(file.batch(1)):
                sequence, target = record
                sequence = sequence.numpy().astype("uint8")
                target = target.numpy().astype("float16")
                
                # assert that genomic locus from sequences BED file matches the sequence in this record
                row = subset_all_sequences.iloc[total_rec_count]
                sequence_from_fasta = fasta_utils.get_sequence(row["chromosome"], \
                                                               row["start"] + 1, \
                                                               row["end"] + 1, \
                                                               ".", \
                                                               fasta_extractor)
                sequence_from_fasta = np.array([_ for _ in sequence_from_fasta])
                sequence_from_one_hot = ind_to_base[sequence[0].argmax(axis=-1)]
                N_mask = sequence_from_fasta == "N"
                assert np.all(sequence_from_one_hot[~N_mask] == sequence_from_fasta[~N_mask])
                
                # now get expanded sequence
                # left pad if needed
                expanded_sequence_start = max(1, row["start"] + 1 - extra_seq_needed)
                if expanded_sequence_start == 1:
                    left_pad = "".join(["N" for _ in range(1 - (row["start"] + 1 - extra_seq_needed))])
                else:
                    left_pad = ""                    
                
                # right pad if needed
                chr_size = chromosome_sizes[chromosome_sizes["chromosome"] == row["chromosome"]].iloc[0]["size"]
                expanded_sequence_end = min(chr_size + 1, row["end"] + 1 + extra_seq_needed)
                if expanded_sequence_end == (chr_size + 1):
                    right_pad = "".join(["N" for _ in range((row["end"] + 1 + extra_seq_needed) - (chr_size + 1))])
                else:
                    right_pad = ""                
                
                expanded_sequence = fasta_utils.get_sequence(row["chromosome"], \
                                                             expanded_sequence_start, \
                                                             expanded_sequence_end, \
                                                             ".", \
                                                             fasta_extractor)
                expanded_sequence = left_pad + expanded_sequence + right_pad
                assert len(expanded_sequence) == enformer_input_size
                
                # one-hot encode it
                expanded_sequence = np.array([i.upper() for i in expanded_sequence])
                expanded_sequence_1hot = np.zeros((len(expanded_sequence), 4), dtype="uint8")
                expanded_sequence_1hot[expanded_sequence == "A", 0] = 1
                expanded_sequence_1hot[expanded_sequence == "C", 1] = 1
                expanded_sequence_1hot[expanded_sequence == "G", 2] = 1
                expanded_sequence_1hot[expanded_sequence == "T", 3] = 1
                expanded_sequence_1hot = expanded_sequence_1hot.reshape(1, len(expanded_sequence), 4)
                                
                if not check:
                    new_file.write(serialize_example(expanded_sequence_1hot, target))
                
                total_rec_count += 1

human
train
/clusterfs/nilah/richard/basenji2/data_pretrained_basenji2/human/tfrecords/train-0-0.tfr


2024-02-18 14:18:34.956718: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22997 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:1a:00.0, compute capability: 7.5
2024-02-18 14:18:34.958205: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22997 MB memory:  -> device: 1, name: NVIDIA TITAN RTX, pci bus id: 0000:1b:00.0, compute capability: 7.5
2024-02-18 14:18:34.959403: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 22997 MB memory:  -> device: 2, name: NVIDIA TITAN RTX, pci bus id: 0000:60:00.0, compute capability: 7.5
2024-02-18 14:18:34.960569: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 22997 MB memory:  -> device: 3, name: NVIDIA TITAN RTX, pci bus id: 0000:61:00.0, 

/clusterfs/nilah/richard/basenji2/data_pretrained_basenji2/human/tfrecords/train-0-1.tfr


256it [00:44,  5.72it/s]


/clusterfs/nilah/richard/basenji2/data_pretrained_basenji2/human/tfrecords/train-0-2.tfr


...

256it [00:22, 11.44it/s]


/clusterfs/nilah/richard/basenji2/data_pretrained_basenji2/mouse/tfrecords/valid-1-7.tfr


256it [00:21, 11.81it/s]


/clusterfs/nilah/richard/basenji2/data_pretrained_basenji2/mouse/tfrecords/valid-1-8.tfr


161it [00:14, 11.24it/s]
